In [1]:

import os
import re
import random
import pprint
import mne
from mne.io.edf.edf import RawEDF
import numpy as np
from datetime import datetime
import pandas as pd


In [20]:
rootdir = f'{os.getcwd()}/data/'
patient = 'chb01'

file_path = f'{rootdir}{patient}/{patient}-summary.txt'

In [21]:
file_metadata = []
with open(file_path) as f:
    content_str = f.read()
    regex = re.compile(r'^\Z|\*+') # match empty string or literal asterisks
    filtered = [x for x in content_str.split('\n') if not regex.search(x)]
    regex = re.compile('Channel \d+') # match channel numbers
    # channels = [x.split(':')[-1].strip() for x in filtered if regex.search(x)]
    regex = re.compile('Data Sampling Rate:')
    # fs = int([x.split(':')[-1].strip() for x in filtered if regex.search(x)][0].split(' ')[0])
    regex = re.compile('^(?!Channel|Data).') # match file names
    file_metas = [x for x in filtered if regex.findall(x)]
    file_meta = {}
    for x in file_metas:
        k, v = x.partition(':')[::2]

        if k == 'Seizure Start Time':
            file_meta['Seizure Start Time'] = v
        if k == 'Seizure End Time':
            file_meta['Seizure End Time'] = v
            tup_meta = {'File Name': file_meta['File Name'], 
                            'File Start Time': file_meta['File Start Time'], 
                            'File End Time': file_meta['File End Time'],
                            'Number of Seizures in File': file_meta['Number of Seizures in File'],
                            'Seizure Start Time': file_meta['Seizure Start Time'],
                            'Seizure End Time': file_meta['Seizure End Time']
                        }
            file_metadata.append(tup_meta)

        if k == 'File Name':
            file_meta['File Name'] = v.strip()
        if k == 'File Start Time':
            file_meta['File Start Time'] = v.strip()
        if k == 'File End Time':
            file_meta['File End Time'] = v.strip()
        if k == 'Number of Seizures in File':
            if int(v) == 0:
                if 'Seizure End Time' in file_meta:
                    del file_meta['Seizure End Time']
                if 'Seizure Start Time' in file_meta:
                    del file_meta['Seizure Start Time']
                file_meta['Number of Seizures in File'] = 0
                tup_meta = {'File Name': file_meta['File Name'], 
                            'File Start Time': file_meta['File Start Time'], 
                            'File End Time': file_meta['File End Time'],
                            'Number of Seizures in File': file_meta['Number of Seizures in File']
                        }
                file_metadata.append(tup_meta)
            if int(v) > 0:
                file_meta['Number of Seizures in File'] = int(v.strip())

In [24]:
df = pd.DataFrame({"Seizure File": [], "Seizure Start":[], "Seizure End":[]})
temp_ss = []
temp_es = []
temp_file = []

for i in range(len(file_metadata)):
    
    if file_metadata[i]['Number of Seizures in File'] > 0:
        temp_ss.append(int(re.findall('\d+', file_metadata[i]['Seizure Start Time'])[-1]))
        temp_es.append(int(re.findall('\d+', file_metadata[i]['Seizure End Time'])[-1]))
        temp_file.append(file_metadata[i]['File Name'].replace('.edf',''))
        
        
for i in range(len(temp_ss)):
    df.loc[len(df)] = [temp_file[i], temp_ss[i], temp_es[i]]      

df['Seizure Duration'] = df.apply(lambda x: x['Seizure End'] - x['Seizure Start'], axis=1)
df

,Seizure File,Seizure Start,Seizure End,Seizure Duration
0,chb01_03,2996.0,3036.0,40.0
1,chb01_04,1467.0,1494.0,27.0
2,chb01_15,1732.0,1772.0,40.0
3,chb01_16,1015.0,1066.0,51.0
4,chb01_18,1720.0,1810.0,90.0
5,chb01_21,327.0,420.0,93.0
6,chb01_26,1862.0,1963.0,101.0


In [28]:
df['Seizure Start'][1]

1467.0

In [29]:
for i in range(df.shape[0]):
    marker_start = df['Seizure Start'][i]
    marker_end = df['Seizure End'][i]
    data = mne.io \
            .read_raw_edf(input_fname=file_path, preload=False, verbose='Error') \
            .crop(tmin=marker_start, tmax=_marker_end) \
            .get_data(picks='all', units='uV', return_times=False)
    savename = filename.split('.')[0] + 'ictal' + '.npy'
    savepath = root + case + savename
    np.save(savepath, data)

NameError: name 'filepath' is not defined

'chb01_03'

In [58]:
import pandas as pd
df = pd.DataFrame({"Seizure Start":[], "Seizure End":[]})
temp_ss = []
temp_es = []

for line in lines:

    if re.findall('Seizure Start Time', line):
        temp_ss.append(int(re.findall('\d+', line)[-1]))
        
    if re.findall('Seizure End Time', line):
        temp_es.append(int(re.findall('\d+', line)[-1])) 


for i in range(len(temp_ss)):
    df.loc[len(df)] = [temp_ss[i], temp_es[i]]       


df['Seizure Duration'] = df.apply(lambda x: x['Seizure End'] - x['Seizure Start'], axis=1)
df


,Seizure Start,Seizure End,Seizure Duration
0,2996.0,3036.0,40.0
1,1467.0,1494.0,27.0
2,1732.0,1772.0,40.0
3,1015.0,1066.0,51.0
4,1720.0,1810.0,90.0
5,327.0,420.0,93.0
6,1862.0,1963.0,101.0


In [14]:
file_metadata[1]

{'File Name': 'chb01_02.edf',
 'File Start Time': '12:42:57',
 'File End Time': '13:42:57',
 'Number of Seizures in File': 0}